<pre>
Copyright 2021-2022 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
import nest_asyncio
from isabelle_client import start_isabelle_server

nest_asyncio.apply()
# on Windows one should start Isabelle server using Cygwin
# and paste `server_info` string here
server_info, _ = start_isabelle_server()

In [2]:
%%time
# check that any lattice axiom is independent from all the rest
from residuated_binars.constants import LATTICE
from residuated_binars.use_nitpick import use_nitpick
from residuated_binars.utils import remove_dirs

remove_dirs(("task*", "hyp*"))
max_dim = 10
use_nitpick(
    max_cardinality=max_dim,
    independent_assumptions=[
        # this is needed to skip the lattice axioms check
        # what we are trying to find here are not real lattices
        axiom.replace("join", "joan")
        for axiom in LATTICE
    ],
    additional_assumptions=[],
    check_subset_independence=False,
    server_info=server_info
)

CPU times: user 63.4 ms, sys: 1.39 ms, total: 64.8 ms
Wall time: 1min 15s


In [3]:
# in this package we have also a parser for Isabelle server responses
import os
from residuated_binars.parser import isabelle_response_to_algebra

structures = []
for i in range(2, max_dim):
    try:
        structures += isabelle_response_to_algebra(
            os.path.join(".", f"task{i}", "isabelle.out")
        )
    except:
        pass

In [4]:
# let's double check that structres found really respect all but one axiom each
from residuated_binars.axiom_checkers import commutative, associative, absorbs

for some in structures:
    score = ""
    score += "1" if commutative(some.operations["joan"]) else "0"
    score += "1" if commutative(some.operations["meet"]) else "0"
    score += "1" if associative(some.operations["joan"]) else "0"
    score += "1" if associative(some.operations["meet"]) else "0"
    score += "1" if absorbs(some.operations["meet"], some.operations["joan"]) else "0"
    score += "1" if absorbs(some.operations["joan"], some.operations["meet"]) else "0"
    print(score)    

111110
111101
011111
101111
110111
111011
